In [1]:
import os
import h5py
import itertools

import numpy as np
import pandas as pd

from tqdm import tqdm

In [2]:
def all_pairs(lst):
    if len(lst) < 2:
        yield []
        return
    if len(lst) % 2 == 1:
        # Handle odd length list
        for i in range(len(lst)):
            for result in all_pairs(lst[:i] + lst[i+1:]):
                yield result
    else:
        a = lst[0]
        for i in range(1,len(lst)):
            pair = (a,lst[i])
            for rest in all_pairs(lst[1:i]+lst[i+1:]):
                yield [pair] + rest

def Mjets(jets):
    # jets: 一個形狀為 (n, 4) 的 NumPy 陣列，其中 n 是噴射數量，每個噴射有四個屬性（pt, eta, phi, m）

    pt, eta, phi, m = jets.T  # 將噴射屬性分解為單獨的陣列

    px = pt * np.cos(phi)
    py = pt * np.sin(phi)
    pz = pt * np.sinh(eta)
    e = np.sqrt(m*m + px*px + py*py + pz*pz)

    return np.sqrt(e.sum()**2 - px.sum()**2 - py.sum()**2 - pz.sum()**2)

In [3]:
def get_Higgs_result(total_event, total_Higgs, correct_event, correct_Higgs, nh, nj):

    start_nj, end_nj = nj

    label = ['all' if nh == 'all' else f'{nh}h' for nj in range(start_nj, end_nj+2)]

    if nh == 'all':
        nh = slice(1, None)
        
    event_type = [f'Nj={nj}' for nj in range(start_nj, end_nj)]
    event_type.append(f'Nj>={end_nj}')
    event_type.append('Total')

    event_fraction = [total_event[nh, nj].sum() / total_event.sum() for nj in range(start_nj, end_nj)]
    event_fraction.append(total_event[nh, end_nj:].sum() / total_event.sum())
    event_fraction.append(total_event[nh].sum() / total_event.sum())

    event_efficiency = [correct_event[nh, nj].sum() / total_event[nh, nj].sum() for nj in range(start_nj, end_nj)]
    event_efficiency.append(correct_event[nh, end_nj:].sum() / total_event[nh, end_nj:].sum())
    event_efficiency.append(correct_event[nh].sum() / total_event[nh].sum())

    higgs_efficiency = [correct_Higgs[nh, nj].sum() / total_Higgs[nh, nj].sum() for nj in range(start_nj, end_nj)]
    higgs_efficiency.append(correct_Higgs[nh, end_nj:].sum() / total_Higgs[nh, end_nj:].sum())
    higgs_efficiency.append(correct_Higgs[nh].sum() / total_Higgs[nh].sum())
        
    result = {'Label':label,
              'Event type': event_type,
              'Event Fraction': event_fraction,
              'Event Efficiency': event_efficiency,
              'Higgs Efficiency': higgs_efficiency,
             }

    df = pd.DataFrame(result)

    return df

In [4]:
def chi2_triHiggs(m1, m2, m3):
    mh = 125.0
    return (m1 - mh)**2 + (m2 - mh)**2 + (m3 - mh)**2

def compare_jet_list_triHiggs(pair1, pair2, nh_max=3):
    h1_true = {pair1[0], pair1[1]}
    h2_true = {pair1[2], pair1[3]}
    h3_true = {pair1[4], pair1[5]}  
    
    h1_test = {pair2[0], pair2[1]}
    h2_test = {pair2[2], pair2[3]}
    h3_test = {pair2[4], pair2[5]}
    
    test_h = [h1_test, h2_test, h3_test]
    
    nh = 0
    for id1, id2, id3 in itertools.permutations([0, 1, 2]):
        h1 = test_h[id1]
        h2 = test_h[id2]
        h3 = test_h[id3]
        if h1_true == h1:
            nh = 1
            if h2_true == h2:
                nh = 2
                if h3_true == h3:
                    nh = 3
                    break
                    
    same = True if nh==nh_max else False
    return same, nh

In [5]:
def get_particle_mask(quark_jet, particle_quarks):
    # quark_jet: 每個夸克對應的 jet 編號，shape 為 (n_event, 6)
    # particle_quarks: 粒子對應的夸克編號，shape 為 (n_quarks,)

    # 檢查是否每個夸克都有對應的 jet
    mask1 = np.all(quark_jet[:, particle_quarks] != -1, axis=1)

    # 對每一個事件，檢查每個夸克對應的 jet 都不重複
    count = np.array([[np.sum(event == event[i]) for i in particle_quarks] for event in quark_jet])
    mask2 = np.all(count == 1, axis=1)

    return mask1 & mask2

def chi2_method_triHiggs(file_path, use_btag=False):
    MAX_JETS = 15

    with h5py.File(file_path, 'r') as f:
        
        total_event = np.zeros((4, MAX_JETS + 1))
        total_Higgs = np.zeros((4, MAX_JETS + 1))
        correct_event = np.zeros((4, MAX_JETS + 1))
        correct_Higgs = np.zeros((4, MAX_JETS + 1))

        nevent = f['INPUTS/Source/pt'].shape[0]
        nevent = 1000

        for event in tqdm(range(nevent)):

            nj = f['INPUTS/Source/MASK'][event].sum()
            pt = f['INPUTS/Source/pt'][event]
            eta = f['INPUTS/Source/eta'][event]
            phi = f['INPUTS/Source/phi'][event]
            mass = f['INPUTS/Source/mass'][event]
            btag = f['INPUTS/Source/btag'][event]

            
            h1_b1 = f['TARGETS/h1/b1'][event] 
            h1_b2 = f['TARGETS/h1/b2'][event] 
            h2_b1 = f['TARGETS/h2/b1'][event] 
            h2_b2 = f['TARGETS/h2/b2'][event] 
            h3_b1 = f['TARGETS/h3/b1'][event] 
            h3_b2 = f['TARGETS/h3/b2'][event] 

            quark_jet = np.array([h1_b1, h1_b2, h2_b1, h2_b2, h3_b1, h3_b2]).reshape(1, 6)

            h1_mask = get_particle_mask(quark_jet, [0, 1])
            h2_mask = get_particle_mask(quark_jet, [2, 3])
            h3_mask = get_particle_mask(quark_jet, [4, 5])
            
            is_3h_event = h1_mask and h2_mask and h3_mask
            is_2h_event = (~h1_mask and h2_mask and h3_mask) or (h1_mask and ~h2_mask and h3_mask) or (h1_mask and h2_mask and ~h3_mask)
            is_1h_event = (~h1_mask and ~h2_mask and h3_mask) or (~h1_mask and h2_mask and ~h3_mask) or (h1_mask and ~h2_mask and ~h3_mask)
            is_0h_event = ~h1_mask and ~h2_mask and ~h3_mask
            
            if is_3h_event:
                total_event[3, nj] += 1
                total_Higgs[3, nj] += 3
            if is_2h_event:
                total_event[2, nj] += 1
                total_Higgs[2, nj] += 2   
            if is_1h_event:
                total_event[1, nj] += 1
                total_Higgs[1, nj] += 1   
            if is_0h_event:
                total_event[0, nj] += 1
                continue

            true_pair = [h1_b1,h1_b2, h2_b1,h2_b2, h3_b1,h3_b2]

            chisq = -1 
            pair = []

            jets_index = np.where(btag)[0] if use_btag else range(nj)

            for combination in itertools.combinations(jets_index, 6):
                for (i1,i2), (i3,i4), (i5,i6) in all_pairs(combination):       
                    jets = np.array([[pt[i], eta[i], phi[i], mass[i]] for i in [i1, i2, i3, i4, i5, i6]])

                    mh1, mh2, mh3 = Mjets(jets[[0, 1]]), Mjets(jets[[2, 3]]), Mjets(jets[[4, 5]])

                    tem = chi2_triHiggs(mh1, mh2, mh3)

                    if chisq < 0 or tem < chisq:
                        chisq = tem
                        pair = [i1, i2, i3, i4, i5, i6]

                        
            if is_3h_event:
                same, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=3)
                correct_Higgs[3, nj] += nh
                if same: correct_event[3, nj] += 1 
            if is_2h_event:
                same, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=2)
                correct_Higgs[2, nj] += nh
                if same: correct_event[2, nj] += 1     
            if is_1h_event:
                same, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=1)
                correct_Higgs[1, nj] += nh
                if same: correct_event[1, nj] += 1

        Nj = (6, 8)
        
        df_all = get_Higgs_result(total_event, total_Higgs, correct_event, correct_Higgs, nh='all', nj=Nj)
        print('All Events:')
        print(df_all)
        df_1h = get_Higgs_result(total_event, total_Higgs, correct_event, correct_Higgs, nh=1, nj=Nj)
        print('1 Higgs Events:')
        print(df_1h)
        df_2h = get_Higgs_result(total_event, total_Higgs, correct_event, correct_Higgs, nh=2, nj=Nj)
        print('2 Higgs Events:')
        print(df_2h)
        df_3h = get_Higgs_result(total_event, total_Higgs, correct_event, correct_Higgs, nh=3, nj=Nj)
        print('3 Higgs Events:')
        print(df_3h)
        
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        
        frames = [df_all, df_1h, df_2h, df_3h]
        result = pd.concat(frames)
        
        result.to_csv(f'{file_name}--chisq_result.csv')

In [6]:
chi2_method_triHiggs('/home/r10222035/SPANet2/data/triHiggs/gghhh_0b_test.h5', use_btag=False)

100%|██████████| 1000/1000 [00:27<00:00, 35.78it/s]

All Events:
  Label Event type  Event Fraction  Event Efficiency  Higgs Efficiency
0   all       Nj=6           0.396          0.131313          0.228198
1   all       Nj=7           0.242          0.078512          0.166667
2   all      Nj>=8           0.198          0.035354          0.085106
3   all      Total           0.836          0.093301          0.174403
1 Higgs Events:
  Label Event type  Event Fraction  Event Efficiency  Higgs Efficiency
0    1h       Nj=6           0.181          0.104972          0.104972
1    1h       Nj=7           0.097          0.092784          0.092784
2    1h      Nj>=8           0.072          0.069444          0.069444
3    1h      Total           0.350          0.094286          0.094286
2 Higgs Events:
  Label Event type  Event Fraction  Event Efficiency  Higgs Efficiency
0    2h       Nj=6           0.138          0.014493          0.123188
1    2h       Nj=7           0.088          0.011364          0.096591
2    2h      Nj>=8           0.07

In [6]:
def get_Higgs_correct_fraction(events, nh, nj):
    # events: number of events in different categories (nh, nj, n_correct_h)
    start_nj, end_nj = nj

    label = [f'{nh}h' for _ in range(start_nj, end_nj+2)]

    total_event = events[nh].sum(axis=1)

    correct_3h_event = events[nh, :, 3]
    correct_2h_event = events[nh, :, 2]
    correct_1h_event = events[nh, :, 1]
    correct_0h_event = events[nh, :, 0]

    correct_Higgs = events[nh, :, 3] * 3 + events[nh, :, 2] * 2 + events[nh, :, 1]
        
    event_type = [f'Nj={nj}' for nj in range(start_nj, end_nj)]
    event_type.append(f'Nj>={end_nj}')
    event_type.append('Total')


    eff_3h = [correct_3h_event[nj] / total_event[nj] for nj in range(start_nj, end_nj)]
    eff_3h.append(correct_3h_event[end_nj:].sum() / total_event[end_nj:].sum())
    eff_3h.append(correct_3h_event.sum() / total_event.sum())

    eff_2h = [correct_2h_event[nj] / total_event[nj] for nj in range(start_nj, end_nj)]
    eff_2h.append(correct_2h_event[end_nj:].sum() / total_event[end_nj:].sum())
    eff_2h.append(correct_2h_event.sum() / total_event.sum())

    eff_1h = [correct_1h_event[nj] / total_event[nj] for nj in range(start_nj, end_nj)]
    eff_1h.append(correct_1h_event[end_nj:].sum() / total_event[end_nj:].sum())
    eff_1h.append(correct_1h_event.sum() / total_event.sum())

    eff_0h = [correct_0h_event[nj] / total_event[nj] for nj in range(start_nj, end_nj)]
    eff_0h.append(correct_0h_event[end_nj:].sum() / total_event[end_nj:].sum())
    eff_0h.append(correct_0h_event.sum() / total_event.sum())

    eff_Higgs = [correct_Higgs[nj] / (total_event[nj] * nh) for nj in range(start_nj, end_nj)]
    eff_Higgs.append(correct_Higgs[end_nj:].sum() / (total_event[end_nj:].sum() * nh))
    eff_Higgs.append(correct_Higgs.sum() / (total_event.sum() * nh))
        
    result = {'Label':label,
              'Event type': event_type,
              '3h': eff_3h,
              '2h': eff_2h,
              '1h': eff_1h,
              '0h': eff_0h,
              'Higgs': eff_Higgs
             }

    df = pd.DataFrame(result)

    return df

def chi2_method_triHiggs_2(file_path, use_btag=False):
    MAX_JETS = 15

    with h5py.File(file_path, 'r') as f:
        
        # events: 總共有多少該類事件 (nh, nj, n_correct_h)
        events = np.zeros((4, MAX_JETS + 1, 4))

        nevent = f['INPUTS/Source/pt'].shape[0]
        nevent = 1000

        for event in tqdm(range(nevent)):

            nj = f['INPUTS/Source/MASK'][event].sum()
            pt = f['INPUTS/Source/pt'][event]
            eta = f['INPUTS/Source/eta'][event]
            phi = f['INPUTS/Source/phi'][event]
            mass = f['INPUTS/Source/mass'][event]
            btag = f['INPUTS/Source/btag'][event]

            
            h1_b1 = f['TARGETS/h1/b1'][event] 
            h1_b2 = f['TARGETS/h1/b2'][event] 
            h2_b1 = f['TARGETS/h2/b1'][event] 
            h2_b2 = f['TARGETS/h2/b2'][event] 
            h3_b1 = f['TARGETS/h3/b1'][event] 
            h3_b2 = f['TARGETS/h3/b2'][event] 

            quark_jet = np.array([h1_b1, h1_b2, h2_b1, h2_b2, h3_b1, h3_b2]).reshape(1, 6)

            h1_mask = get_particle_mask(quark_jet, [0, 1])
            h2_mask = get_particle_mask(quark_jet, [2, 3])
            h3_mask = get_particle_mask(quark_jet, [4, 5])
            
            is_3h_event = h1_mask and h2_mask and h3_mask
            is_2h_event = (~h1_mask and h2_mask and h3_mask) or (h1_mask and ~h2_mask and h3_mask) or (h1_mask and h2_mask and ~h3_mask)
            is_1h_event = (~h1_mask and ~h2_mask and h3_mask) or (~h1_mask and h2_mask and ~h3_mask) or (h1_mask and ~h2_mask and ~h3_mask)
            is_0h_event = ~h1_mask and ~h2_mask and ~h3_mask

            true_pair = [h1_b1,h1_b2, h2_b1,h2_b2, h3_b1,h3_b2]

            chisq = -1 
            pair = []

            jets_index = np.where(btag)[0] if use_btag else range(nj)

            for combination in itertools.combinations(jets_index, 6):
                for (i1,i2), (i3,i4), (i5,i6) in all_pairs(combination):       
                    jets = np.array([[pt[i], eta[i], phi[i], mass[i]] for i in [i1, i2, i3, i4, i5, i6]])

                    mh1, mh2, mh3 = Mjets(jets[[0, 1]]), Mjets(jets[[2, 3]]), Mjets(jets[[4, 5]])

                    tem = chi2_triHiggs(mh1, mh2, mh3)

                    if chisq < 0 or tem < chisq:
                        chisq = tem
                        pair = [i1, i2, i3, i4, i5, i6]

                        
            if is_3h_event:
                _, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=3)
                events[3, nj, nh] += 1
            if is_2h_event:
                _, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=2)
                events[2, nj, nh] += 1   
            if is_1h_event:
                _, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=1)
                events[1, nj, nh] += 1
            if is_0h_event:
                events[0, nj, 0] += 1

        Nj = (6, 8)
        
        df_1h = get_Higgs_correct_fraction(events, nh=1, nj=Nj)
        print('1 Higgs Events:')
        print(df_1h)
        df_2h = get_Higgs_correct_fraction(events, nh=2, nj=Nj)
        print('2 Higgs Events:')
        print(df_2h)
        df_3h = get_Higgs_correct_fraction(events, nh=3, nj=Nj)
        print('3 Higgs Events:')
        print(df_3h)
        
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        
        frames = [df_1h, df_2h, df_3h]
        result = pd.concat(frames)
        
        result.to_csv(f'{file_name}--chisq_result.csv')

def SPANet_pairing_triHiggs(true_file, predict_file, nevent_max=10000):
    MAX_JETS = 15

    with h5py.File(true_file, 'r') as f_true, h5py.File(predict_file, 'r') as f_pre:
        
        # events: 總共有多少該類事件 (nh, nj, n_correct_h)
        events = np.zeros((4, MAX_JETS + 1, 4))

        nevent = min(f_true['INPUTS/Source/pt'].shape[0], nevent_max)

        for event in tqdm(range(nevent)):

            nj = f_true['INPUTS/Source/MASK'][event].sum()

            h1_b1 = f_true['TARGETS/h1/b1'][event] 
            h1_b2 = f_true['TARGETS/h1/b2'][event] 
            h2_b1 = f_true['TARGETS/h2/b1'][event] 
            h2_b2 = f_true['TARGETS/h2/b2'][event] 
            h3_b1 = f_true['TARGETS/h3/b1'][event] 
            h3_b2 = f_true['TARGETS/h3/b2'][event]

            quark_jet = np.array([h1_b1, h1_b2, h2_b1, h2_b2, h3_b1, h3_b2]).reshape(1, 6)

            h1_mask = get_particle_mask(quark_jet, [0, 1])
            h2_mask = get_particle_mask(quark_jet, [2, 3])
            h3_mask = get_particle_mask(quark_jet, [4, 5])
            
            is_3h_event = h1_mask and h2_mask and h3_mask
            is_2h_event = (~h1_mask and h2_mask and h3_mask) or (h1_mask and ~h2_mask and h3_mask) or (h1_mask and h2_mask and ~h3_mask)
            is_1h_event = (~h1_mask and ~h2_mask and h3_mask) or (~h1_mask and h2_mask and ~h3_mask) or (h1_mask and ~h2_mask and ~h3_mask)
            is_0h_event = ~h1_mask and ~h2_mask and ~h3_mask

            true_pair = [h1_b1,h1_b2, h2_b1,h2_b2, h3_b1,h3_b2]

            pair = [f_pre['TARGETS/h1/b1'][event],
                    f_pre['TARGETS/h1/b2'][event],
                    f_pre['TARGETS/h2/b1'][event],
                    f_pre['TARGETS/h2/b2'][event],
                    f_pre['TARGETS/h3/b1'][event],
                    f_pre['TARGETS/h3/b2'][event]]

            if is_3h_event:
                _, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=3)
                events[3, nj, nh] += 1
            if is_2h_event:
                _, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=2)
                events[2, nj, nh] += 1   
            if is_1h_event:
                _, nh = compare_jet_list_triHiggs(true_pair, pair, nh_max=1)
                events[1, nj, nh] += 1
            if is_0h_event:
                events[0, nj, 0] += 1

        Nj = (6, 8)
        
        df_1h = get_Higgs_correct_fraction(events, nh=1, nj=Nj)
        print('1 Higgs Events:')
        print(df_1h)
        df_2h = get_Higgs_correct_fraction(events, nh=2, nj=Nj)
        print('2 Higgs Events:')
        print(df_2h)
        df_3h = get_Higgs_correct_fraction(events, nh=3, nj=Nj)
        print('3 Higgs Events:')
        print(df_3h)
        
        file_name = os.path.splitext(os.path.basename(predict_file))[0]
        
        frames = [df_1h, df_2h, df_3h]
        result = pd.concat(frames)
        
        result.to_csv(f'{file_name}--SPANet_result.csv')

In [8]:
chi2_method_triHiggs_2('/home/r10222035/SPANet2/data/triHiggs/gghhh_0b_test.h5', use_btag=False)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:31<00:00, 32.15it/s]

1 Higgs Events:
  Label Event type   3h   2h        1h        0h     Higgs
0    1h       Nj=6  0.0  0.0  0.104972  0.895028  0.104972
1    1h       Nj=7  0.0  0.0  0.092784  0.907216  0.092784
2    1h      Nj>=8  0.0  0.0  0.069444  0.930556  0.069444
3    1h      Total  0.0  0.0  0.094286  0.905714  0.094286
2 Higgs Events:
  Label Event type   3h        2h        1h        0h     Higgs
0    2h       Nj=6  0.0  0.014493  0.217391  0.768116  0.123188
1    2h       Nj=7  0.0  0.011364  0.170455  0.818182  0.096591
2    2h      Nj>=8  0.0  0.027027  0.148649  0.824324  0.101351
3    2h      Total  0.0  0.016667  0.186667  0.796667  0.110000
3 Higgs Events:
  Label Event type        3h        2h        1h        0h     Higgs
0    3h       Nj=6  0.402597  0.000000  0.142857  0.454545  0.450216
1    3h       Nj=7  0.157895  0.070175  0.228070  0.543860  0.280702
2    3h      Nj>=8  0.000000  0.000000  0.230769  0.769231  0.076923
3    3h      Total  0.215054  0.021505  0.193548  0.569892  0

In [9]:
true_file = '/home/r10222035/SPANet2/data/triHiggs/gghhh_0b_test.h5'
predict_file = '/home/r10222035/SPANet2/data/triHiggs/gghhh_0b_test-predict.h5'
SPANet_pairing_triHiggs(true_file, predict_file)

100%|██████████| 52290/52290 [02:27<00:00, 353.85it/s]

1 Higgs Events:
  Label Event type   3h   2h        1h        0h     Higgs
0    1h       Nj=6  0.0  0.0  0.162483  0.837517  0.162483
1    1h       Nj=7  0.0  0.0  0.150334  0.849666  0.150334
2    1h      Nj>=8  0.0  0.0  0.145794  0.854206  0.145794
3    1h      Total  0.0  0.0  0.155971  0.844029  0.155971
2 Higgs Events:
  Label Event type   3h        2h        1h        0h     Higgs
0    2h       Nj=6  0.0  0.054968  0.288859  0.656173  0.199397
1    2h       Nj=7  0.0  0.051811  0.258854  0.689334  0.181239
2    2h      Nj>=8  0.0  0.034583  0.247899  0.717518  0.158533
3    2h      Total  0.0  0.050102  0.271850  0.678048  0.186027
3 Higgs Events:
  Label Event type        3h        2h        1h        0h     Higgs
0    3h       Nj=6  0.532476  0.000000  0.119234  0.348290  0.572220
1    3h       Nj=7  0.344528  0.020845  0.165605  0.469021  0.413627
2    3h      Nj>=8  0.237457  0.021993  0.186254  0.554296  0.314204
3    3h      Total  0.374620  0.014239  0.156423  0.454717  0

In [7]:
true_file = '/home/r10222035/SPANet2/data/triHiggs/triHiggs_0b_test.h5'
predict_file = '/home/r10222035/SPANet2/data/triHiggs/triHiggs_0b_test-predict.h5'
SPANet_pairing_triHiggs(true_file, predict_file)

100%|██████████| 10000/10000 [00:23<00:00, 422.96it/s]


1 Higgs Events:
  Label Event type   3h   2h        1h        0h     Higgs
0    1h       Nj=6  0.0  0.0  0.168159  0.831841  0.168159
1    1h       Nj=7  0.0  0.0  0.171756  0.828244  0.171756
2    1h      Nj>=8  0.0  0.0  0.122257  0.877743  0.122257
3    1h      Total  0.0  0.0  0.161255  0.838745  0.161255
2 Higgs Events:
  Label Event type   3h        2h        1h        0h     Higgs
0    2h       Nj=6  0.0  0.082079  0.280438  0.637483  0.222298
1    2h       Nj=7  0.0  0.061630  0.276342  0.662028  0.199801
2    2h      Nj>=8  0.0  0.035211  0.253521  0.711268  0.161972
3    2h      Total  0.0  0.066535  0.274045  0.659420  0.203557
3 Higgs Events:
  Label Event type        3h        2h        1h        0h     Higgs
0    3h       Nj=6  0.656463  0.000000  0.081633  0.261905  0.683673
1    3h       Nj=7  0.436416  0.017341  0.167630  0.378613  0.503854
2    3h      Nj>=8  0.341727  0.017986  0.172662  0.467626  0.411271
3    3h      Total  0.478214  0.011983  0.141612  0.368192  0